## Kostenfunktionen

In dieser Lektion lernen wir, wie man eine *Kostenfunktion* auswertet:

- Zuerst lernen wir etwas über [Qiskit Runtime-Grundelemente](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/primitives.html)
- Definieren Sie eine *Kostenfunktion* $C(\vec\theta)$. Dies ist eine problemspezifische Funktion, die das Ziel des Problems definiert, das der Optimierer minimieren (oder maximieren) soll.
- Definieren einer Messstrategie mit den Qiskit Runtime-Primitiven zur Optimierung von Geschwindigkeit und Genauigkeit

&nbsp;

![Kostenfunktions-Workflow](images/cost_function_workflow.png)

## Primitive

Alle physikalischen Systeme, ob klassisch oder Quantensysteme, können in verschiedenen Zuständen existieren. Beispielsweise kann ein Auto auf einer Straße eine bestimmte Masse, Position, Geschwindigkeit oder Beschleunigung haben, die seinen Zustand charakterisiert. Ebenso können Quantensysteme auch unterschiedliche Konfigurationen oder Zustände haben, sie unterscheiden sich jedoch von klassischen Systemen in der Art und Weise, wie wir mit Messungen und Zustandsentwicklung umgehen. Dies führt zu einzigartigen Eigenschaften wie *Superposition* und *Verschränkung* , die nur der Quantenmechanik vorbehalten sind. So wie wir den Zustand eines Autos mithilfe physikalischer Eigenschaften wie Geschwindigkeit oder Beschleunigung beschreiben können, können wir den Zustand eines Quantensystems auch mithilfe von *Observablen* beschreiben, bei denen es sich um mathematische Objekte handelt.

In der Quantenmechanik werden Zustände durch normalisierte komplexe Spaltenvektoren oder *Kets* ($|\psi\rangle$) dargestellt und Observablen sind hermitesche lineare Operatoren ($\hat{H}=\hat{H}^{\dagger}$ ), die auf die Kets einwirken. Ein Eigenvektor ($|\lambda\rangle$) einer Observablen wird als *Eigenzustand* bezeichnet. Wenn wir eine Observable für einen ihrer Eigenzustände ($|\lambda\rangle$) messen, erhalten wir den entsprechenden Eigenwert ($\lambda$) als Ergebnis.

Wenn Sie sich fragen, wie man ein Quantensystem misst und was Sie messen können, bietet Qiskit zwei [Grundelemente](gloss:primitives) an, die Ihnen helfen können:

- `Sampler` : Bei einem gegebenen Quantenzustand $|\psi\rangle$ ermittelt dieses Grundelement die Wahrscheinlichkeit jedes möglichen rechnerischen Basiszustands.
- `Estimator` : Bei einer gegebenen Quantenobservable $\hat{H}$ und einem Zustand $|\psi\rangle$ berechnet dieses Grundelement den erwarteten Wert von $\hat{H}$.


### Das Sampler-Grundelement

Das `Sampler` Grundelement berechnet die Wahrscheinlichkeit, jeden möglichen Zustand $|k\rangle$ aus der Berechnungsbasis zu erhalten, vorausgesetzt, es gibt einen Quantenschaltkreis, der den Zustand $|\psi\rangle$ vorbereitet. Es rechnet

$$
p_k = |\langle k | \psi \rangle|^2 \quad \forall k \in \mathbb{Z}_2^n \equiv \{0,1,\cdots,2^n-1\},
$$

Dabei ist $n$ die Anzahl der Qubits und $k$ die ganzzahlige Darstellung einer möglichen Ausgabe-Binärzeichenfolge ${0,1}^n$ (d. h. ganze Zahlen mit der Basis $2$).

`Sampler` [von Qiskit Runtime](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.Sampler.html) führt die Schaltung mehrmals auf einem Quantengerät durch, führt bei jedem Durchlauf Messungen durch und rekonstruiert die Wahrscheinlichkeitsverteilung aus den wiederhergestellten Bitfolgen. Je mehr Läufe (oder *Schüsse* ) es ausführt, desto genauer werden die Ergebnisse sein, aber das erfordert mehr Zeit und Quantenressourcen.

Da jedoch die Anzahl der möglichen Ausgaben exponentiell mit der Anzahl der Qubits $n$ (dh $2^n$) wächst, muss auch die Anzahl der Schüsse exponentiell wachsen, um eine *dichte* Wahrscheinlichkeitsverteilung zu erfassen. Daher ist `Sampler` nur für *spärliche* Wahrscheinlichkeitsverteilungen effizient; wobei der Zielzustand $|\psi\rangle$ als lineare Kombination der rechnerischen Basiszustände ausgedrückt werden muss, wobei die Anzahl der Terme höchstens polynomial mit der Anzahl der Qubits wächst:

$$
|\psi\rangle = \sum^{\text{Poly}(n)}_k w_k |k\rangle.
$$

Der `Sampler` kann auch so konfiguriert werden, dass er Wahrscheinlichkeiten aus einem Teilabschnitt der Schaltung abruft, die eine Teilmenge der gesamten möglichen Zustände darstellen.

### Das Estimator-Grundelement

Das `Estimator` Grundelement berechnet den Erwartungswert einer Observablen $\hat{H}$ für einen Quantenzustand $|\psi\rangle$; wobei die beobachtbaren Wahrscheinlichkeiten als $p_\lambda = |\langle\lambda|\psi\rangle|^2$ ausgedrückt werden können, wobei $|\lambda\rangle$ die Eigenzustände des beobachtbaren $\hat{H}$ sind. Der Erwartungswert wird dann als Durchschnitt aller möglichen Ergebnisse $\lambda$ (d. h. der Eigenwerte der Observablen) einer Messung des Zustands $|\psi\rangle$ definiert, gewichtet mit den entsprechenden Wahrscheinlichkeiten:

$$
\langle\hat{H}\rangle_\psi := \sum_\lambda p_\lambda \lambda = \langle \psi | \hat{H} | \psi \rangle
$$

Allerdings ist die Berechnung des Erwartungswerts einer Observablen nicht immer möglich, da wir ihre Eigenbasis oft nicht kennen. `Estimator` [von Qiskit Runtime](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.Estimator.html) verwendet einen komplexen algebraischen Prozess, um den Erwartungswert eines realen Quantengeräts abzuschätzen, indem er das Observable in eine Kombination anderer Observablen zerlegt, deren Eigenbasis wir kennen.

Einfacher ausgedrückt zerlegt `Estimator` alle Observablen, die er nicht messen kann, in einfachere, messbare Observablen, die [Pauli-Operatoren](gloss:pauli) genannt werden.

Jeder Operator kann als Kombination von $4^n$ Pauli-Operatoren ausgedrückt werden.

$$
\hat{P}_k := 
\sigma_{k_{n-1}}\otimes \cdots \otimes \sigma_{k_0} \quad 
\forall k \in \mathbb{Z}_4^n \equiv \{0,1,\cdots,4^n-1\}, \\
$$

so dass

$$
\hat{H} = \sum^{4^n-1}_{k=0} w_k \hat{P}_k
$$

wobei $n$ die Anzahl der Qubits ist, $k \equiv k_{n-1} \cdots k_0$ für $k_l \in \mathbb{Z}_4 \equiv {0, 1, 2, 3}$ (also ganze Zahlen). Basis $4$) und $(\sigma_0, \sigma_1, \sigma_2, \sigma_3) := (I, X, Y, Z)$.

Nach Durchführung dieser Zerlegung leitet `Estimator` einen neuen Schaltkreis $V_k|\psi\rangle$ für jede Observable $\hat{P}_k$ ab (dh aus dem ursprünglichen Schaltkreis), um die Pauli-Observable in der Berechnungsbasis effektiv *zu diagonalisieren* und zu messen . Wir können Pauli-Observablen leicht messen, weil wir $V_k$ im Voraus kennen, was bei anderen Observablen im Allgemeinen nicht der Fall ist.

Für jedes `{latex} \hat{P}_{k}` führt der `Estimator` die entsprechende Schaltung auf einem Quantengerät mehrmals aus, misst den Ausgabezustand auf der Berechnungsbasis und berechnet die Wahrscheinlichkeit $p_{kj}$ für den Erhalt jedes einzelnen mögliche Ausgabe $j$. Anschließend wird nach dem Eigenwert $\lambda_{kj}$ von $P_k$ gesucht, der jeder Ausgabe $j$ entspricht, mit $w_k$ multipliziert und alle Ergebnisse addiert, um den erwarteten Wert der Observablen $\hat{H zu erhalten }$ für den gegebenen Zustand $|\psi\rangle$.

$$
\langle\hat{H}\rangle_\psi = 
\sum_{k=0}^{4^n-1} w_k \sum_{j=0}^{2^n-1}p_{kj} \lambda_{kj},
$$

Da die Berechnung des Erwartungswerts von $4^n$ Paulis unpraktisch ist (d. h. exponentiell wachsend), kann `Estimator` nur dann effizient sein, wenn eine große Menge von $w_k$ Null ist (d. h. *spärliche* Pauli-Zerlegung statt *dicht* ). Formal sagen wir, dass die Anzahl der von Null verschiedenen Terme höchstens polynomiell mit der Anzahl der Qubits $n$ wachsen muss, damit diese Berechnung *effizient lösbar* ist: $\hat{H} = \sum^{\text{Poly }(n)}_k w_k \hat{P}_k.$

Dem Leser fällt möglicherweise die implizite Annahme auf, dass die [Wahrscheinlichkeitsstichprobe](gloss:sampling) ebenfalls effizient sein muss, wie für `Sampler` erklärt, was bedeutet

$$
\langle\hat{H}\rangle_\psi = 
\sum_{k}^{\text{Poly}(n)} w_k \sum_{j}^{\text{Poly}(n)}p_{kj} \lambda_{kj}.
$$

### Geführtes Beispiel zur Berechnung von Erwartungswerten

Nehmen wir den Einzel-Qubit-Zustand $|+\rangle := H|0\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$ und beobachtbar an

$$
\begin{aligned}
\hat{H}
& = \begin{pmatrix} 
-1 & 2 \\
2 & -1 \\
\end{pmatrix}\\[1mm]
& = 2X - Z

\end{aligned}
$$

mit folgendem theoretischen Erwartungswert $\langle\hat{H}\rangle_+ = \langle+|\hat{H}|+\rangle = 2.$

Da wir nicht wissen, wie wir diese Observable messen sollen, können wir ihren Erwartungswert nicht direkt berechnen und müssen ihn als $\langle\hat{H}\rangle_+ = 2\langle X \rangle_+ - \langle neu ausdrücken Z \rangle_+ $. Es kann gezeigt werden, dass es zum gleichen Ergebnis führt, indem man feststellt, dass $\langle+|X|+\rangle = 1$ und $\langle+|Z|+\rangle = 0$ ist.

Sehen wir uns an, wie man $\langle X \rangle_+$ und $\langle Z \rangle_+$ direkt berechnet. Da $X$ und $Z$ nicht kommutieren (d. h. nicht die gleiche Eigenbasis haben), können sie nicht gleichzeitig gemessen werden, daher benötigen wir die Hilfsschaltkreise:

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp

# The following code will work for any other initial single-qubit state and observable
original_circuit = QuantumCircuit(1)
original_circuit.h(0)

H = SparsePauliOp(["X", "Z"], [2, -1])

aux_circuits = []
for pauli in H.paulis:
    aux_circ = original_circuit.copy()
    aux_circ.barrier()
    if str(pauli) == "X":
        aux_circ.h(0)
    elif str(pauli) == "Y":
        aux_circ.sdg(0)
        aux_circ.h(0)
    else:
        aux_circ.i(0)
    aux_circ.measure_all()
    aux_circuits.append(aux_circ)


print("Original circuit:")
display(original_circuit.draw("mpl"))
for (circuit, pauli) in zip(aux_circuits, H.paulis):
    print(f"Auxiliary circuit for {str(pauli)}")
    display(circuit.draw("mpl"))

Wir können nun die Berechnung manuell mit `Sampler` durchführen und die Ergebnisse mit `Estimator` überprüfen:

In [ ]:
from qiskit.primitives import Sampler, Estimator
from qiskit.circuit.library import IGate, ZGate
import numpy as np


## SAMPLER
sampler = Sampler()
job = sampler.run(aux_circuits)
probability_dists = job.result().quasi_dists

expvals = []
for dist, pauli in zip(probability_dists, H.paulis):
    val = 0
    if str(pauli) == "I":
        Lambda = IGate().to_matrix().real
    else:
        Lambda = ZGate().to_matrix().real
    val += Lambda[0][0] * dist.get(0, 0)
    val += Lambda[1][1] * dist.get(1, 0)
    expvals.append(val)


print("Sampler results:")
for (pauli, expval) in zip(H.paulis, expvals):
    print(f"  >> Expected value of {str(pauli)}: {expval:.5f}")

total_expval = np.sum(H.coeffs * expvals).real
print(f"  >> Total expected value: {total_expval:.5f}")


## ESTIMATOR
observables = [
    *H.paulis,
    H,
]  # Note: run for individual Paulis as well as full observable H

estimator = Estimator()
job = estimator.run([original_circuit] * len(observables), observables)
estimator_expvals = job.result().values

print("Estimator results:")
for (obs, expval) in zip(observables, estimator_expvals):
    if obs is not H:
        print(f"  >> Expected value of {str(obs)}: {expval:.5f}")
    else:
        print(f"  >> Total expected value: {expval:.5f}")

### Mathematische Genauigkeit (optional)

Wenn man $|\psi\rangle$ in Bezug auf die Basis der Eigenzustände von $\hat{H}$ ausdrückt, $|\psi\rangle = \sum_\lambda a_\lambda |\lambda\rangle$, folgt:

$$
\begin{aligned}
\langle \psi | \hat{H} | \psi \rangle
& = \bigg(\sum_{\lambda'}a^*_{\lambda'} \langle \lambda'|\bigg) \hat{H} 
  \bigg(\sum_{\lambda} a_\lambda | \lambda\rangle\bigg)\\[1mm]

& = \sum_{\lambda}\sum_{\lambda'} a^*_{\lambda'}a_{\lambda} 
  \langle \lambda'|\hat{H}| \lambda\rangle\\[1mm]

& = \sum_{\lambda}\sum_{\lambda'} a^*_{\lambda'}a_{\lambda} \lambda 
\langle \lambda'| \lambda\rangle\\[1mm]

& = \sum_{\lambda}\sum_{\lambda'} a^*_{\lambda'}a_{\lambda} \lambda 
\cdot \delta_{\lambda, \lambda'}\\[1mm]

& = \sum_\lambda |a_\lambda|^2 \lambda\\[1mm]

& = \sum_\lambda p_\lambda \lambda\\[1mm]

\end{aligned}
$$

Da wir die Eigenwerte oder Eigenzustände der Zielobservablen $\hat{H}$ nicht kennen, müssen wir zunächst ihre Diagonalisierung betrachten. Vorausgesetzt, dass $\hat{H}$ [hermitesch](gloss:hermitian) ist, gibt es eine einheitliche Transformation $V$ mit $\hat{H}=V^\dagger \Lambda V,$, wobei $\Lambda$ die diagonale Eigenwertmatrix ist, also $\langle j | \Lambda | k \rangle = 0$, wenn $j\neq k$ und $\langle j | \Lambda | j \rangle = \lambda_j$.

Dies impliziert, dass der erwartete Wert wie folgt umgeschrieben werden kann:

$$
\begin{aligned}
\langle\psi|\hat{H}|\psi\rangle
& = \langle\psi|V^\dagger \Lambda V|\psi\rangle\\[1mm]

& = \langle\psi|V^\dagger \bigg(\sum_{j=0}^{2^n-1} |j\rangle 
\langle j|\bigg) \Lambda \bigg(\sum_{k=0}^{2^n-1} |k\rangle \langle k|\bigg) V|\psi\rangle\\[1mm]

& = \sum_{j=0}^{2^n-1} \sum_{k=0}^{2^n-1}\langle\psi|V^\dagger |j\rangle 
\langle j| \Lambda  |k\rangle \langle k| V|\psi\rangle\\[1mm]

& = \sum_{j=0}^{2^n-1}\langle\psi|V^\dagger |j\rangle 
\langle j| \Lambda  |j\rangle \langle j| V|\psi\rangle\\[1mm]

& = \sum_{j=0}^{2^n-1}|\langle j| V|\psi\rangle|^2 \lambda_j\\[1mm]

\end{aligned}
$$

Vorausgesetzt, wenn sich ein System im Zustand $|\phi\rangle = V |\psi\rangle$ befindet, beträgt die Wahrscheinlichkeit der Messung $| j\rangle$ ist $p_j = |\langle j|\phi \rangle|^2$, der oben erwartete Wert kann ausgedrückt werden als:

$$
\langle\psi|\hat{H}|\psi\rangle = 
\sum_{j=0}^{2^n-1} p_j \lambda_j.
$$

Es ist sehr wichtig zu beachten, dass die Wahrscheinlichkeiten aus dem Zustand $V |\psi\rangle$ und nicht aus $|\psi\rangle$ entnommen werden. Deshalb ist die Matrix $V$ unbedingt notwendig.

Sie fragen sich vielleicht, wie Sie die Matrix $V$ und die Eigenwerte $\Lambda$ erhalten. Wenn Sie die Eigenwerte bereits hätten, wäre es nicht nötig, einen Quantencomputer zu verwenden, da das Ziel von Variationsalgorithmen darin besteht, diese Eigenwerte von $\hat{H}$ zu finden.

Glücklicherweise gibt es einen Weg, das zu umgehen: Jede $2^n \times 2^n$-Matrix kann als lineare Kombination von $4^n$-Tensorprodukten von $n$ Pauli-Matrizen und -Identitäten geschrieben werden, die alle sowohl hermitesch als auch hermitesch sind einheitlich mit bekannten $V$ und $\Lambda$. Dies ist, was `Estimator` von Runtime intern tut, indem er jedes [Operator-](https://qiskit.org/documentation/stubs/qiskit.quantum_info.Operator.html) Objekt in ein [SparsePauliOp](https://qiskit.org/documentation/stubs/qiskit.quantum_info.SparsePauliOp.html) zerlegt.

Hier sind die Operatoren, die verwendet werden können:

$$
\begin{array}{c|c|c|c}
  \text{Operator} & \sigma & V & \Lambda \\[1mm]
  \hline
  I & \sigma_0 = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} & V_0 = I & \Lambda_0 = I = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} \\[4mm]

  X & \sigma_1 = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} & V_1 = H =\frac{1}{\sqrt{2}} \begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix} & \Lambda_1 = \sigma_3 = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} \\[4mm]

  Y & \sigma_2 = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix} & V_2 = HS^\dagger  =\frac{1}{\sqrt{2}} \begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}\cdot  \begin{pmatrix} 1 & 0 \\ 0 & -i \end{pmatrix} = \frac{1}{\sqrt{2}} \begin{pmatrix} 1 & -i \\ 1 & i \end{pmatrix}\quad & \Lambda_2 = \sigma_3 = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} \\[4mm]

  Z & \sigma_3 = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} & V_3 = I & \Lambda_3 = \sigma_3 = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}
\end{array}
$$

Schreiben wir also $\hat{H}$ in Bezug auf die Paulis und Identitäten um:

$$
\hat{H} = 
\sum_{k_{n-1}=0}^3...
\sum_{k_0=0}^3 w_{k_{n-1}...k_0} 
\sigma_{k_{n-1}}\otimes ... \otimes \sigma_{k_0} = \sum_{k=0}^{4^n-1} w_k \hat{P}_k,
$$

wobei $k = \sum_{l=0}^{n-1} 4^l k_l \equiv k_{n-1}...k_0$ für $k_{n-1},...,k_0\in {0,1,2,3}$ (also Basis $4$) und `{latex} \hat{P}_{k} := \sigma_{k_{n-1}}\otimes ... \otimes \sigma_{k_0}` :

$$
\begin{aligned}
\langle\psi|\hat{H}|\psi\rangle
& = \sum_{k=0}^{4^n-1} w_k 
\sum_{j=0}^{2^n-1}|\langle j| V_k|\psi\rangle|^2 \langle j| \Lambda_k |j\rangle \\[1mm]

& = \sum_{k=0}^{4^n-1} w_k \sum_{j=0}^{2^n-1}p_{kj} \lambda_{kj}, \\[1mm]


\end{aligned}
$$

wobei $V_k := V_{k_{n-1}}\otimes ... \otimes V_{k_0}$ und $\Lambda_k := \Lambda_{k_{n-1}}\otimes ... \otimes \ Lambda_{k_0}$, so dass: $\hat{P_k}=V_k^\dagger \Lambda_k V_k.$

## Kostenfunktionen

Im Allgemeinen werden Kostenfunktionen verwendet, um das Ziel eines Problems zu beschreiben und um zu beschreiben, wie gut ein Testzustand in Bezug auf dieses Ziel abschneidet. Diese Definition kann auf verschiedene Beispiele aus den Bereichen Chemie, maschinelles Lernen, Finanzen, Optimierung usw. angewendet werden.

Betrachten wir ein einfaches Beispiel für die Ermittlung des Grundzustands eines Systems. Unser Ziel ist es, den Erwartungswert der Observablen zu minimieren, die die Energie darstellen (Hamiltonian $\hat{\mathcal{H}}$):

$$
\min_{\vec\theta} \langle\psi(\vec\theta)|\hat{\mathcal{H}}|\psi(\vec\theta)\rangle
$$

Wir können das [*Estimator-* Grundelement](https://github.com/qiskit-community/prototype-zne/blob/main/docs/tutorials/0-estimator.ipynb) verwenden, um den Erwartungswert auszuwerten und diesen Wert zur Minimierung an einen Optimierer zu übergeben. Wenn die Optimierung erfolgreich ist, wird eine Reihe optimaler Parameterwerte $\vec\theta^ *$ zurückgegeben, aus denen wir den vorgeschlagenen Lösungszustand $|\psi(\vec\theta^ )\rangle$ und konstruieren können* Berechnen Sie den beobachteten Erwartungswert als $C(\vec\theta^*)$.

Beachten Sie, dass wir die Kostenfunktion nur für die begrenzte Anzahl von Zuständen minimieren können, die wir berücksichtigen. Dies führt uns zu zwei getrennten Möglichkeiten:

- **Unser Ansatz definiert nicht den Lösungszustand im gesamten Suchraum** : Wenn dies der Fall ist, wird unser Optimierer nie die Lösung finden und wir müssen mit anderen Ansätzen experimentieren, die unseren Suchraum möglicherweise genauer darstellen können.
- **Unser Optimierer kann diese gültige Lösung nicht finden** : Die Optimierung kann global und lokal definiert werden. Wir werden im späteren Abschnitt untersuchen, was dies bedeutet.

Alles in allem werden wir eine klassische Optimierungsschleife durchführen, uns jedoch auf die Auswertung der Kostenfunktion an einen Quantencomputer verlassen. Aus dieser Perspektive könnte man sich die Optimierung als ein rein klassisches Unterfangen vorstellen, bei dem wir jedes Mal, wenn der Optimierer die Kostenfunktion bewerten muss, ein [*Black-Box-Quantenorakel*](gloss:oracle) aufrufen.

In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator

# Add your token below
service = QiskitRuntimeService(channel="ibm_quantum")

def cost_function_vqe(theta):
    observable = SparsePauliOp.from_list([("XX", 1), ("YY", -3)])

    reference_circuit = QuantumCircuit(2)
    reference_circuit.x(0)

    variational_form = TwoLocal(
        2,
        rotation_blocks=["rz", "ry"],
        entanglement_blocks="cx",
        entanglement="linear",
        reps=1,
    )
    ansatz = reference_circuit.compose(variational_form)

    backend = service.backend("ibmq_qasm_simulator")
    
    # Use estimator to get the expected values corresponding to each ansatz
    estimator = Estimator(session=backend)
    job = estimator.run(ansatz, observable, theta)
    values = job.result().values

    return values


theta_list = (2 * np.pi * np.random.rand(1, 8)).tolist()
cost_function_vqe(theta_list)

### Beispiel-Mapping auf nicht-physikalische Systeme

Das Maximum-Cut-Problem (Max-Cut) ist ein kombinatorisches Optimierungsproblem, bei dem die Eckpunkte eines Graphen in zwei disjunkte Mengen so aufgeteilt werden, dass die Anzahl der Kanten zwischen den beiden Mengen maximiert wird. Formeller ausgedrückt verlangt das Max-Cut-Problem bei gegebenem ungerichteten Graphen $G=(V,E)$, wobei $V$ die Menge der Eckpunkte und $E$ die Menge der Kanten ist, die Eckpunkte in zwei disjunkte Teilmengen zu unterteilen , $S$ und $T$, so dass die Anzahl der Kanten mit einem Endpunkt in $S$ und dem anderen in $T$ maximiert wird.

Wir können Max-Cut anwenden, um verschiedene Probleme zu lösen, darunter Clustering, Netzwerkdesign, Phasenübergänge usw. Wir beginnen mit der Erstellung eines Problemdiagramms:

In [ ]:
import networkx as nx

n = 4
G = nx.Graph()
G.add_nodes_from(range(n))
edge_list = [(0, 1, 1.0), (0, 2, 1.0), (0, 3, 1.0), (1, 2, 1.0), (2, 3, 1.0)]
G.add_weighted_edges_from(edge_list)

colors = ["red" for i in range(n)]


def draw_graph(G, colors):
    """Draws the graph with the chose colors"""
    layout = nx.shell_layout(G)
    nx.draw_networkx(G, node_color=colors, pos=layout)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=edge_labels)


draw_graph(G, colors)

Dieses Problem kann als binäres Optimierungsproblem ausgedrückt werden. Für jeden Knoten $0 \leq i &lt; n$, wobei $n$ die Anzahl der Knoten des Graphen ist (in diesem Fall $n=4$), betrachten wir die binäre Variable $x_i$. Diese Variable hat den Wert $1$, wenn der Knoten $i$ zu einer der Gruppen gehört, die wir mit $1$ bezeichnen, und $0$, wenn er sich in der anderen Gruppe befindet, die wir mit $0$ bezeichnen. Wir werden auch als $w_{ij}$ (Element $(i,j)$ der Adjazenzmatrix $w$) das Gewicht der Kante bezeichnen, die vom Knoten $i$ zum Knoten $j$ verläuft. Da der Graph ungerichtet ist, gilt $w_{ij}=w_{ji}$. Dann können wir unser Problem so formulieren, dass wir die folgende Kostenfunktion maximieren:

$$
\begin{aligned}
C(\vec{x})
& =\sum_{i,j=0}^n w_{ij} x_i(1-x_j)\\[1mm]

& = \sum_{i,j=0}^n w_{ij} x_i - \sum_{i,j=0}^n w_{ij} x_ix_j\\[1mm]

& = \sum_{i,j=0}^n w_{ij} x_i - \sum_{i=0}^n \sum_{j=0}^i 2w_{ij} x_ix_j
\end{aligned}
$$

Um dieses Problem mit einem Quantencomputer zu lösen, werden wir die Kostenfunktion als den erwarteten Wert einer Observablen ausdrücken. Die von Qiskit anerkannten Observablen bestehen jedoch ursprünglich aus Pauli-Operatoren, die die Eigenwerte $1$ und $-1$ anstelle von $0$ und $1$ haben. Deshalb werden wir folgende Variablenänderung vornehmen:

Wobei $\vec{x}=(x_0,x_1,\cdots ,x_{n-1})$. Wir können die Adjazenzmatrix $w$ verwenden, um bequem auf die Gewichte aller Kanten zuzugreifen. Dies wird verwendet, um unsere Kostenfunktion zu erhalten:

$$
z_i = 1-2x_i \rightarrow x_i = \frac{1-z_i}{2}
$$

Dies impliziert Folgendes:

$$
\begin{array}{lcl} x_i=0 & \rightarrow & z_i=1 \\ x_i=1 & \rightarrow & z_i=-1.\end{array}
$$

Die neue Kostenfunktion, die wir maximieren möchten, ist also:

$$
\begin{aligned}
C(\vec{z})
& = \sum_{i,j=0}^n w_{ij} \bigg(\frac{1-z_i}{2}\bigg)\bigg(1-\frac{1-z_j}{2}\bigg)\\[1mm]

& = \sum_{i,j=0}^n \frac{w_{ij}}{4} - \sum_{i,j=0}^n \frac{w_{ij}}{4} z_iz_j\\[1mm]

& = \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} -  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} z_iz_j
\end{aligned}
$$

Darüber hinaus besteht die natürliche Tendenz eines Quantencomputers darin, Minima (normalerweise die niedrigste Energie) anstelle von Maxima zu finden. Anstatt also $C(\vec{z})$ zu maximieren, werden wir Folgendes minimieren:

$$
-C(\vec{z}) =  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} z_iz_j -  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2}
$$

Da wir nun eine Kostenfunktion zum Minimieren haben, deren Variablen die Werte $-1$ und $1$ haben können, können wir die folgende Analogie zum Pauli-$Z$ ziehen:

$$
z_i \equiv Z_i = \overbrace{I}^{n-1}\otimes ... \otimes \overbrace{Z}^{i} \otimes ... \otimes \overbrace{I}^{0}
$$

Mit anderen Worten, die Variable $z_i$ entspricht einem $Z$-Gate, das auf das Qubit $i$ wirkt. Darüber hinaus:

$$
Z_i|x_{n-1}\cdots x_0\rangle = z_i|x_{n-1}\cdots x_0\rangle \rightarrow \langle x_{n-1}\cdots x_0 |Z_i|x_{n-1}\cdots x_0\rangle = z_i
$$

Dann ist die Observable, die wir betrachten werden:

$$
\hat{H} = \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} Z_iZ_j
$$

zu dem wir anschließend den unabhängigen Term hinzufügen müssen:

$$
\texttt{offset} = - \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2}
$$

Diese Transformation kann mit [`QuadraticProgram.to_ising()`](https://qiskit.org/documentation/optimization/stubs/qiskit_optimization.QuadraticProgram.to_ising.html) durchgeführt werden.

In [ ]:
from qiskit_optimization.applications import Maxcut
from qiskit.algorithms.optimizers import COBYLA

w = nx.to_numpy_array(G)

max_cut = Maxcut(w)
quadratic_program = max_cut.to_quadratic_program()
observable, offset = quadratic_program.to_ising()


def cost_function_max_cut_vqe(theta):

    ansatz = TwoLocal(
        observable.num_qubits, "rx", reps=1
    )

    backend = service.backend("ibmq_qasm_simulator")

    # Use estimator to get the expected values corresponding to each ansatz
    estimator = Estimator(session=backend)
    job = estimator.run(ansatz, observable, theta)
    values = job.result().values

    return values


optimizer = COBYLA()

initial_theta = np.ones(8)

optimizer_result = optimizer.minimize(fun=cost_function_max_cut_vqe, x0=initial_theta)

optimal_parameters = optimizer_result.x
print(optimal_parameters)

Wir werden dieses Beispiel in „Anwendungen“ erneut betrachten, um zu untersuchen, wie ein Optimierer zum Durchlaufen des Suchraums genutzt werden kann. Im Allgemeinen umfasst dies:

- Nutzen Sie einen Optimierer, um optimale Parameter zu finden
- Binden optimaler Parameter an den Ansatz, um die Eigenwerte zu finden
- Übersetzen der Eigenwerte in unsere Problemdefinition

## Messstrategie: Geschwindigkeit vs. Genauigkeit

Wie bereits erwähnt, verwenden wir einen verrauschten Quantencomputer als *Black-Box-Orakel* , bei dem Rauschen die abgerufenen Werte nicht deterministisch machen kann, was zu zufälligen Schwankungen führt, die wiederum die Konvergenz bestimmter Optimierer beeinträchtigen oder sogar vollständig verhindern einen Lösungsvorschlag. Dies ist ein allgemeines Problem, das wir angehen müssen, während wir uns schrittweise dem Quantenvorteil nähern:

![Vorteil](images/cost_function_path_to_quantum_advantage.png)

Wir können die Fehlerunterdrückungs- und Fehlerminderungsoptionen von Qiskit Runtime Primitive nutzen, um Rauschen zu bekämpfen und den Nutzen heutiger Quantencomputer zu maximieren.

### Fehlerunterdrückung

[Unter Fehlerunterdrückung](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/error-suppression.html) versteht man Techniken zur Optimierung und Transformation einer Schaltung während der Kompilierung, um Fehler zu minimieren. Hierbei handelt es sich um eine grundlegende Fehlerbehandlungstechnik, die in der Regel zu einem gewissen klassischen [Vorverarbeitungsaufwand](gloss:overhead) für die Gesamtlaufzeit führt. Der Overhead umfasst das Transpilieren von Schaltkreisen zur Ausführung auf Quantenhardware durch:

- Ausdrücken der Schaltung mithilfe der nativen Gatter, die in einem Quantensystem verfügbar sind
- Zuordnung der virtuellen Qubits zu physischen Qubits
- Hinzufügen von SWAPs basierend auf den Konnektivitätsanforderungen
- Optimierung von 1Q- und 2Q-Gattern
- Hinzufügen einer dynamischen Entkopplung zu inaktiven Qubits, um die Auswirkungen der Dekohärenz zu verhindern.

Primitive ermöglichen die Verwendung von [Techniken zur Fehlerunterdrückung](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/error-suppression.html) , indem die Option `optimization_level` festgelegt und erweiterte Transpilationsoptionen ausgewählt werden. In einem späteren Kurs werden wir uns mit verschiedenen Schaltungskonstruktionsmethoden befassen, um die Ergebnisse zu verbessern. In den meisten Fällen würden wir jedoch empfehlen, `optimization_level=3` festzulegen.

### Fehlerminderung

[Fehlerminderung](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/error-mitigation.html) bezieht sich auf Techniken, die es Benutzern ermöglichen, Schaltungsfehler zu reduzieren, indem sie das Geräterauschen zum Zeitpunkt der Ausführung modellieren. Typischerweise führt dies zu einem Quanten-Vorverarbeitungs-Overhead im Zusammenhang mit dem Modelltraining und einem klassischen Nachverarbeitungs-Overhead, um Fehler in den Rohergebnissen durch die Verwendung des generierten Modells zu mindern.

Die Option `resilience_level` des Qiskit Runtime-Primitivs gibt den Grad der Widerstandsfähigkeit an, die gegen Fehler aufgebaut werden soll. Höhere Ebenen liefern genauere Ergebnisse auf Kosten längerer Verarbeitungszeiten aufgrund des Quanten-Sampling-Overheads. Belastbarkeitsstufen können verwendet werden, um den Kompromiss zwischen Kosten und Genauigkeit zu konfigurieren, wenn Sie Fehlerminderung auf Ihre primitive Abfrage anwenden.

Bei der Implementierung einer Technik zur Fehlerminderung gehen wir davon aus, dass die [Verzerrung](gloss:bias) unserer Ergebnisse im Vergleich zur vorherigen, uneingeschränkten Verzerrung verringert wird. In einigen Fällen kann die Voreingenommenheit sogar verschwinden. Allerdings ist dies mit Kosten verbunden. Wenn wir die Verzerrung unserer geschätzten Mengen verringern, nimmt die statistische Variabilität zu (d. h. die Varianz), was wir durch eine weitere Erhöhung der Anzahl der Schüsse pro Kreis in unserem Stichprobenverfahren berücksichtigen können. Dies führt zu einem Mehraufwand, der über den zur Reduzierung der Verzerrung erforderlichen Aufwand hinausgeht und daher nicht standardmäßig durchgeführt wird. Wir können diesem Verhalten ganz einfach zustimmen, indem wir die Anzahl der Schüsse pro Runde in „options.executions.shots“ anpassen, wie im folgenden Beispiel gezeigt.

![Bias-Varianz-Kompromiss](images/cost_function_bias_variance_trade_off.png)

In diesem Kurs werden wir diese Fehlerminderungsmodelle auf hoher Ebene untersuchen, um die Fehlerminderung zu veranschaulichen, die Qiskit Runtime-Grundelemente durchführen können, ohne dass vollständige Implementierungsdetails erforderlich sind.

### Twirled-Readout-Fehlerauslöschung (T-REx)

Twirled Readout Error Extinction (T-REx) nutzt eine als Pauli-Twirling bekannte Technik, um das während des Prozesses der Quantenmessung entstehende Rauschen zu reduzieren. Diese Technik setzt keine spezifische Form von Rauschen voraus, was sie sehr allgemein und effektiv macht.

Gesamtworkflow:

1. Erfassen Sie Daten für den Nullzustand mit zufälligen Bit-Flips (Pauli X vor der Messung)
2. Erfassen Sie Daten für den gewünschten (verrauschten) Zustand mit zufälligen Bit-Flips (Pauli X vor der Messung).
3. Berechnen Sie die Sonderfunktion für jeden Datensatz und dividieren Sie.

&nbsp;

![T-REX](images/cost_function_trex_data_collection.png)

Wir können dies mit `options.resilience_level = 1` festlegen, wie im folgenden Beispiel gezeigt.

### Extrapolation ohne Rauschen

Bei der Zero-Noise-Extrapolation (ZNE) wird zunächst das Rauschen in der Schaltung, die den gewünschten Quantenzustand vorbereitet, verstärkt, Messungen für mehrere unterschiedliche Rauschpegel erhalten und diese Messungen verwendet, um auf das rauschfreie Ergebnis zu schließen.

Gesamtworkflow:

1. Verstärken Sie das Schaltungsrauschen für mehrere Rauschfaktoren
2. Betreiben Sie alle rauschverstärkten Schaltkreise
3. Extrapolieren Sie zurück zur Nullgeräuschgrenze

&nbsp;

![ZNE](images/cost_function_zne_stages.png)

Wir können dies mit `options.resilience_level = 2` festlegen. Wir können dies weiter optimieren, indem wir eine Vielzahl von `noise_factors` , `noise_amplifiers` und `extrapolators` untersuchen, dies liegt jedoch außerhalb des Rahmens dieses Kurses. Wir empfehlen Ihnen, mit diesen [Optionen wie hier beschrieben](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/error-mitigation.html#advanced-resilience-options) zu experimentieren.

### Wahrscheinlichkeitsfehlerbeseitigung

Proben zur probabilistischen Fehlerunterdrückung (PEC) für eine Sammlung von Schaltkreisen, die im Durchschnitt einen rauschinvertierenden Kanal nachahmen, um das Rauschen in der gewünschten Berechnung zu unterdrücken. Dieser Vorgang ähnelt in etwa der Funktionsweise von Kopfhörern mit Geräuschunterdrückung und führt zu großartigen Ergebnissen. Sie ist jedoch nicht so allgemeingültig wie andere Methoden und der Stichprobenaufwand ist exponentiell.

Gesamtworkflow:

![PEC](images/cost_function_pec_layers.png)

Schritt 1: Pauli Twirling

![PEC Twirling](images/cost_function_pec_pauli_twirling.png)

Schritt 2: Wiederholen Sie die Ebene und lernen Sie das Geräusch kennen

![PEC-Lernen](images/cost_function_pec_learn_layer.png)

Schritt 3: Ableiten einer Wiedergabetreue (Fehler für jeden Rauschkanal)

![PEC-Treue](images/cost_function_pec_curve_fitting.png)

Jede Methode bringt ihren eigenen Overhead mit sich: ein Kompromiss zwischen der Anzahl der benötigten Quantenberechnungen (Zeit) und der Genauigkeit unserer Ergebnisse:

$$
\begin{array}{c|c|c|c}
  \text{Methods} & R=1 \text{, T-REx} & R=2 \text{, ZNE} & R=3 \text{, PEC} \\[1mm]
  \hline
  \text{Assumptions} & \text{None} & \text{Ability to scale noise} & \text{Full knowledge of noise} \\[1mm]
  \text{Qubit overhead} & 1 & 1 & 1 \\[1mm]
  \text{Sampling overhead} & 2 & N_{\text{noise-factors}} & \mathcal{O}(e^{\lambda N_{layers}}) \\[1mm]
  \text{Bias} & 0 & \mathcal{O}(\lambda^{N_{\text{noise-factors}}}) & 0 \\[1mm]
\end{array}
$$

### Verwendung der Abschwächungs- und Unterdrückungsoptionen von Qiskit Runtime

Hier erfahren Sie, wie Sie einen Erwartungswert berechnen und dabei Fehlerminderung und -unterdrückung in Qiskit Runtime verwenden. Dieser Prozess findet während einer Optimierungsschleife mehrmals statt. Wir haben das Beispiel jedoch einfach gehalten, um zu veranschaulichen, wie Fehlerminderung und -unterdrückung konfiguriert werden.

In [ ]:
from qiskit.circuit import Parameter, QuantumCircuit
from qiskit.quantum_info import SparsePauliOp

theta = Parameter("theta")

qc = QuantumCircuit(2)
qc.x(1)
qc.h(0)
qc.cp(theta, 0, 1)
qc.h(0)
observables = SparsePauliOp.from_list([("ZZ", 1)])

qc.draw("mpl")

In [ ]:
## Setup phases
import numpy as np

phases = np.linspace(0, 2 * np.pi, 50)

# phases need to be expressed as a list of lists in order to work
individual_phases = [[phase] for phase in phases]

In [ ]:
## Create noise model
from qiskit.providers.fake_provider import FakeManila
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import Options


# Make a noise model
fake_backend = FakeManila()
noise_model = NoiseModel.from_backend(fake_backend)

# Set options to include the noise model
options = Options()
options.simulator = {
    "noise_model": noise_model,
    "basis_gates": fake_backend.configuration().basis_gates,
    "coupling_map": fake_backend.configuration().coupling_map,
    "seed_simulator": 42,
}

In [ ]:
from qiskit_ibm_runtime import Session, QiskitRuntimeService, Estimator

# Add your token below
service = QiskitRuntimeService(channel="ibm_quantum")

noisy_exp_values = []
exp_values_with_em_es = []

with Session(service=service, backend="ibmq_qasm_simulator") as session:
    # generate noisy results
    estimator = Estimator(options=options)
    job = estimator.run(
        circuits=[qc] * len(phases),
        parameter_values=individual_phases,
        observables=[observables] * len(phases),
        shots=1000,
    )
    result = job.result()
    noisy_exp_values = result.values

    options.optimization_level = 2
    options.resilience_level = 1

    # leverage mitigation and suppression
    estimator = Estimator(options=options)
    job = estimator.run(
        circuits=[qc] * len(phases),
        parameter_values=individual_phases,
        observables=[observables] * len(phases),
        shots=1000,
    )
    result = job.result()
    exp_values_with_em_es = result.values

    session.close()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(phases, noisy_exp_values, "o", label="Noisy")
plt.plot(phases, exp_values_with_em_es, "o", label="Mitigation + Suppression")
plt.plot(phases, 2 * np.sin(phases / 2) ** 2 - 1, label="Theory")
plt.xlabel("Phase")
plt.ylabel("Expectation")
plt.legend()
plt.show()

In dieser Lektion haben Sie gelernt, wie Sie eine Kostenfunktion erstellen:

- Erstellen Sie eine Kostenfunktion
- So nutzen Sie [Qiskit Runtime-Grundelemente](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/primitives.html) , um Rauschen zu mildern und zu unterdrücken
- So definieren Sie eine Messstrategie zur Optimierung von Geschwindigkeit und Genauigkeit

Hier ist unser variierendes Arbeitspensum auf hohem Niveau:

![Kostenfunktionsschaltung](images/cost_function_circuit.png)

Unsere Kostenfunktion läuft während jeder Iteration der Optimierungsschleife. In der nächsten Lektion wird untersucht, wie der klassische Optimierer unsere Kostenfunktionsbewertung verwendet, um neue Parameter auszuwählen.